In [2]:
#In this notebook I'm using the current world records as a base for our formula.
from flask import Flask, render_template, request, redirect
import requests
from bs4 import BeautifulSoup
import matplotlib
import scipy as sp
#%matplotlib inline
import math
import matplotlib.pyplot as plt
import numpy as np
from pylab import *
from scipy.optimize import curve_fit
from bokeh.embed import components
from bokeh.plotting import figure
from bokeh.resources import INLINE
from bokeh.templates import RESOURCES
from bokeh.util.string import encode_utf8


app = Flask(__name__)

app.vars = {}

app.vars['color'] = {
    'Male Senior': 'navy',
    #'Female Senior': 'orange'
}

#Index page
@app.route('/')
def main():
    return redirect('/index')

#Error page
@app.route('/error-page')
def error_quandle():
    return render_template('error.html')

#Collecting from index
@app.route('/index', methods=['GET', 'POST'])
def index():
    if request.method == 'GET':
        return render_template('index.html')
    else:
        app.vars['userbw'] = request.form['userbw']
        app.vars['usertotal'] = request.form['usertotal']
        app.vars['features'] = request.form.getlist('features')

        if app.vars['userbw'].strip() == '' or app.vars['usertotal'] == '':
            return redirect('/error-page')

        #return ('/results')
        # Pull stock data
        #url = 'https://www.quandl.com/api/v3/datasets/WIKI/' + app.vars['ticker'] + '/data.json'
        #r = requests.get(url)
        #if r.status_code == 404:
            #return redirect('/error-quandle')
        else:
            wr = [(55.62, 305), (61.81, 332), (68.68, 359), (76.40, 380), (84.69, 394), (93.52, 418), (104.76, 436), (147.48,472)]
    
    bw = []
    total = []

    for i in range(len(wr)):
        bw.append(wr[i][0])
        total.append(wr[i][1])
    
    x = np.array(bw)
    y = np.array(total)

    def func(x, a, b, c, d):
        return a*np.log(c*(x-b))+d

    popt, pcov = curve_fit(func, x, y, [10,50,0.001,0])
            #data = r.json()['dataset_data']['data']
            #cols = r.json()['dataset_data']['column_names']
            #app.vars['data'] = pd.DataFrame(data, columns=cols)
            #app.vars['data'] = df[['Date'] + app.vars['features']].head(5)
            return redirect('/results')
        
@app.route('/results', methods=['GET'])
def graph():
    userbw = app.vars['userbw']
    usertotal = app.vars['usertotal']
    
    #df = app.vars['data']

    # Create a line plot from our data.
    
    
    plt.plot(bw,total, "ro")
    plt.plot(userbw,usertotal, "bo")
    x=linspace(50,190,100)
    plot(x,func(x,*popt))
    plt.xlabel('Body Weight')
    plt.ylabel('World Records')
    plt.savefig('approxlog2.png')
    #plt.savefig('approxlog2.pdf')
    #show()
    
    
    #p = figure(width=700, height=500, x_axis_type="datetime",
                #title="Data from Quandle WIKI set")
    #for category in app.vars['features']:
        #p.line(pd.to_datetime(df['Date']), df[category],
                #color=app.vars['color'][category], line_width=1,
                #legend=app.vars['ticker'] + ": " + category)

    #p.legend.orientation = "top_right"

    # Configure resources to include BokehJS inline in the document.
    plot_resources = RESOURCES.render(
        js_raw=INLINE.js_raw,
        css_raw=INLINE.css_raw,
        js_files=INLINE.js_files,
        css_files=INLINE.css_files,
    )

    script, div = components(approx.png, INLINE)
    html = render_template(
        'results.html',
        #ticker=app.vars['ticker'],
        plot_script=script, plot_div=div, plot_resources=plot_resources
    )
    return encode_utf8(html)        
            
if __name__ == '__main__':
    #app.run('0.0.0.0')
    app.run(port=33507)            